사용 라이브러리 초기화

In [18]:
import numpy as np
import matplotlib.pyplot as plt
import copy
import math
from tqdm import tqdm

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.utils import to_categorical

JSON 파일 읽어오기

In [19]:
import json

#JSON 파일 경로
json_path = "C:\\Users\\USER\\Downloads\\tictactoe_data_complete.json" # 파일경로

#JSON 불러오기
with open(json_path, 'r') as f:
    data = json.load(f)

# 내부 데이터 추출
game_data = data["game_data"]

# 데이터 순서 랜덤화
np.random.shuffle(game_data)

# 훈련 데이터 80% / 검증 데이터 20% 분리
split_ratio = 0.8
split_index = int(len(game_data) * split_ratio)

train_data = game_data[:split_index]
test_data = game_data[split_index:]

# board_state 및 action 추출 함수 정의
def extract_state_action(data):
    board_state = []
    action = []
    for item in data:
        board_state.append(item["board_state_before"])  
        action.append(item["action"])             
    return np.array(board_state), np.array(action)

# 변환
input_shape = (4, 4, 1)
num_classes = 16

x_train, y_train = extract_state_action(train_data)
x_test, y_test = extract_state_action(test_data)

x_train = x_train.reshape(-1, 4, 4, 1)
x_test = x_test.reshape(-1, 4, 4, 1)

# y 데이터 원-핫 인코딩
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

# #특정 컬럼 10개씩 출력하는 함수
# def print_column_samples(field_name, num_samples=10):
#     print(f"\n--- {field_name} (Top {num_samples}) ---")
#     for i in range(min(num_samples, len(game_data))):
#         print(f"[{i}] {game_data[i][field_name]}")

# #출력
# print_column_samples("board_state_before")
# print_column_samples("board_state_current")
# print_column_samples("action")
# print_column_samples("player")
# print_column_samples("winner")

In [20]:
class Environment():
    
    def __init__(self):
    # 보드는 0으로 초기화된 16개의 배열로 준비
    # 게임종료 : done = True
        self.board_a = np.zeros(16)
        self.done = False
        self.reward = 0
        self.winner = 0
        self.print = False

    def move(self, p1, p2, player):
    # 각 플레이어가 선택한 행동을 표시 하고 게임 상태(진행 또는 종료)를 판단
    # p1 = 1, p2 = -1로 정의
    # 각 플레이어는 행동을 선택하는 select_action 메서드를 가짐
        if player == 1:
            pos = p1.select_action(env, player)
        else:
            pos = p2.select_action(env, player)
        
        # 보드에 플레이어의 선택을 표시
        self.board_a[pos] = player
        if self.print:
            print(player)
            self.print_board()
        # 게임이 종료상태인지 아닌지를 판단
        self.end_check(player)
        
        return  self.reward, self.done
 
    # 현재 보드 상태에서 가능한 행동(둘 수 있는 장소)을 탐색하고 리스트로 반환
    def get_action(self):
        observation = []
        for i in range(16):
            if self.board_a[i] == 0:
                observation.append(i)
        return observation
    
    # 게임이 종료(승패 또는 비김)됐는지 판단
    def end_check(self,player):
        # 0 1 2 3
        # 4 5 6 7
        # 8 9 10 11
        # 12 13 14 15
        # 승패 조건은 가로, 세로, 대각선 이 -1 이나 1 로 동일할 때 
        end_condition = ((0,1,2,3),(4,5,6,7),(8,9,10,11),(12,13,14,15),(0,5,10,15),(3,6,9,12), (0,4,8,12), (1,5,9,13), (2,6,10,14), (3,7,11,15))
        for line in end_condition:
            if self.board_a[line[0]] == self.board_a[line[1]] \
                and self.board_a[line[1]] == self.board_a[line[2]] \
                and self.board_a[line[2]] == self.board_a[line[3]] \
                and self.board_a[line[0]] != 0:
                # 종료됐다면 누가 이겼는지 표시
                self.done = True
                self.reward = player
                return
        # 비긴 상태는 더는 보드에 빈 공간이 없을때
        observation = self.get_action()
        if (len(observation)) == 0:
            self.done = True
            self.reward = 0            
        return
        
    # 현재 보드의 상태를 표시 p1 = O, p2 = X    
    def print_board(self):
        print("+----+----+----+----+")
        for i in range(4):
            for j in range(4):
                if self.board_a[4*i+j] == 1:
                    print("|  O",end=" ")
                elif self.board_a[4*i+j] == -1:
                    print("|  X",end=" ")
                else:
                    print("|   ",end=" ")
            print("|")
            print("+----+----+----+----+")

In [21]:
class Human_player():
    
    def __init__(self):
        self.name = "Human player"
        
    def select_action(self, env, player):
        while True:
            # 가능한 행동을 조사한 후 표시
            available_action = env.get_action()
            print("possible actions = {}".format(available_action))

            # 상태 번호 표시
            print("+----+----+----+----+")
            print("+  0 +  1 +  2 +  3 +")
            print("+----+----+----+----+")
            print("+  4 +  5 +  6 +  7 +")
            print("+----+----+----+----+")
            print("+  8 +  9 + 10 + 11 +")
            print("+----+----+----+----+")
            print("+ 12 + 13 + 14 + 15 +")
            print("+----+----+----+----+")
                        
            # 키보드로 가능한 행동을 입력 받음
            action = input("Select action(human) : ")
            action = int(action)
            
            # 입력받은 행동이 가능한 행동이면 반복문을 탈출
            if action in available_action:
                return action
            # 아니면 행동 입력을 반복
            else:
                print("You selected wrong action")
        return

합성곱신경망과 완전연결계층 조합

In [22]:
# import keras
# from keras.models import Sequential
# from keras.layers import Dense, Dropout, Flatten
# from keras.layers import Conv2D, MaxPooling2D
# from keras import backend as K

np.random.seed(0)
model = Sequential()

# 합성곱층 추가
model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=input_shape, name='Conv2D_layer'))

# 최대 풀링층 추가
model.add(MaxPooling2D(pool_size=(2, 2), name = 'MaxPooling2D_layer'))

# 플래튼층 추가
model.add(Flatten(name='Flatten_layer'))

# 드롭아웃 적용
#model.add(Dropout(0.3)) 

# 완전연결계층 추가
model.add(Dense(32, activation='relu', name='Dense1_layer'))
model.add(Dense(num_classes, activation='softmax', name='Dense2_layer'))

# 학습 방법 추가
model.compile(loss = keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adadelta(), metrics=['accuracy'])

# 모델 확인
# print(model.summary())

In [23]:
# 배치 사이즈와 에폭 수 정의
batch_size = 32 # 기존 128
epochs = 100

# 학습 전 합성곱신경망 저장
weight_before, bias_before = model.layers[0].get_weights()

# 합성곱신경망 학습
hist = model.fit(x_train, y_train,
                batch_size = batch_size,
                epochs=epochs,
                verbose=2,
                validation_data=(x_test, y_test))

# 학습 후 합성곱신경망 필터 저장
weight_after, bias_after = model.layers[0].get_weights()

# 학습된 합성공 신경망 성능 확인
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss : ', score[0])
print('Test accuracy : ', score[1])

Train on 10004 samples, validate on 2502 samples
Epoch 1/100
 - 1s - loss: 2.7594 - acc: 0.1002 - val_loss: 2.7361 - val_acc: 0.1211
Epoch 2/100
 - 1s - loss: 2.7202 - acc: 0.1186 - val_loss: 2.6948 - val_acc: 0.1287
Epoch 3/100
 - 1s - loss: 2.6947 - acc: 0.1261 - val_loss: 2.6794 - val_acc: 0.1279
Epoch 4/100
 - 1s - loss: 2.6804 - acc: 0.1338 - val_loss: 2.6697 - val_acc: 0.1315
Epoch 5/100
 - 1s - loss: 2.6672 - acc: 0.1429 - val_loss: 2.6591 - val_acc: 0.1427
Epoch 6/100
 - 1s - loss: 2.6572 - acc: 0.1434 - val_loss: 2.6606 - val_acc: 0.1419
Epoch 7/100
 - 1s - loss: 2.6486 - acc: 0.1451 - val_loss: 2.6480 - val_acc: 0.1419
Epoch 8/100
 - 1s - loss: 2.6405 - acc: 0.1514 - val_loss: 2.6451 - val_acc: 0.1435
Epoch 9/100
 - 1s - loss: 2.6325 - acc: 0.1516 - val_loss: 2.6469 - val_acc: 0.1439
Epoch 10/100
 - 1s - loss: 2.6261 - acc: 0.1556 - val_loss: 2.6367 - val_acc: 0.1443
Epoch 11/100
 - 1s - loss: 2.6201 - acc: 0.1530 - val_loss: 2.6380 - val_acc: 0.1543
Epoch 12/100
 - 1s - loss

In [24]:
class CNN_player():
    
    def __init__(self, model):
        self.name = "CNN player"
        self.model = model
        
    def select_action(self, env, player):

        board_2d = env.board_a.reshape(4, 4)
        
        board_input = board_2d.reshape(1, 4, 4, 1)
        
        # 성능 검증
        predictions = self.model.predict(board_input, verbose=0)
        move_probabilities = predictions[0]
        
        # 가능한 행동 확인
        available_actions = env.get_action()
        
        # 가능한 행동들 중에서 확률이 가장 높은 것 선택
        best_action = None

        sorted_actions = np.argsort(move_probabilities)[::-1]
        
        for action in sorted_actions:
            if action in available_actions:
                best_action = action
                break
        
        if best_action is None:
            best_action = np.random.choice(available_actions)
            
        return best_action

실제 대전

In [26]:
np.random.seed(0)

#p1 = Human_player()
p1 = CNN_player(model)

p2 = Human_player()
#p2 = CNN_player(model)

# 지정된 게임 수를 자동으로 두게 할 것인지 한게임씩 두게 할 것인지 결정
# auto = True : 지정된 판수(games)를 자동으로 진행 
# auto = False : 한판씩 진행

auto = False

# auto 모드의 게임수
games = 100

print("pl player : {}".format(p1.name))
print("p2 player : {}".format(p2.name))

# 각 플레이어의 승리 횟수를 저장
p1_score = 0
p2_score = 0
draw_score = 0

if auto: 
    # 자동 모드 실행
    for j in tqdm(range(games)):
        
        np.random.seed(j)
        env = Environment()
        
        for i in range(10000):
            # p1 과 p2가 번갈아 가면서 게임을 진행
            # p1(1) -> p2(-1) -> p1(1) -> p2(-1) ...
            reward, done = env.move(p1,p2,(-1)**i)
            # 게임 종료 체크
            if done == True:
                if reward == 1:
                    p1_score += 1
                elif reward == -1:
                    p2_score += 1
                else:
                    draw_score += 1
                break

else:                
    # 한 게임씩 진행하는 수동 모드
    np.random.seed(1)
    while True:
        
        env = Environment()
        env.print = False
        for i in range(10000):
            reward, done = env.move(p1,p2,(-1)**i)
            env.print_board()
            if done == True:
                if reward == 1:
                    print("winner is p1({})".format(p1.name))
                    p1_score += 1
                elif reward == -1:
                    print("winner is p2({})".format(p2.name))
                    p2_score += 1
                else:
                    print("draw")
                    draw_score += 1
                break
        
        # 최종 결과 출력        
        print("final result")
        env.print_board()

        # 한게임 더?최종 결과 출력 
        answer = input("More Game? (y/n)")

        if answer == 'n':
            break           

print("p1({}) = {} p2({}) = {} draw = {}".format(p1.name, p1_score,p2.name, p2_score,draw_score))

pl player : CNN player
p2 player : Human player
+----+----+----+----+
|    |    |    |  O |
+----+----+----+----+
|    |    |    |    |
+----+----+----+----+
|    |    |    |    |
+----+----+----+----+
|    |    |    |    |
+----+----+----+----+
possible actions = [0, 1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
+----+----+----+----+
+  0 +  1 +  2 +  3 +
+----+----+----+----+
+  4 +  5 +  6 +  7 +
+----+----+----+----+
+  8 +  9 + 10 + 11 +
+----+----+----+----+
+ 12 + 13 + 14 + 15 +
+----+----+----+----+
Select action(human) : 12
+----+----+----+----+
|    |    |    |  O |
+----+----+----+----+
|    |    |    |    |
+----+----+----+----+
|    |    |    |    |
+----+----+----+----+
|  X |    |    |    |
+----+----+----+----+
+----+----+----+----+
|  O |    |    |  O |
+----+----+----+----+
|    |    |    |    |
+----+----+----+----+
|    |    |    |    |
+----+----+----+----+
|  X |    |    |    |
+----+----+----+----+
possible actions = [1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15]